# Fine Tune BERT on Patent dataset

# Installation

In [ ]:
!pip install transformers==4.2.1

# Import

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

# Configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PROJECT_DIR = "/content/drive/MyDrive/patent"

In [ ]:
DATA_DIR = PROJECT_DIR + '/data/'

In [ ]:
TRAIN_FILE = DATA_DIR + 'patent_train.csv'
VALIDATION_FILE = DATA_DIR + 'patent_validation.csv'
TEST_FILE = DATA_DIR + 'patent_test.csv'

In [ ]:
MODEL_NAME = 'bert-base-uncased'
OUTPUT_NAME = 'patent_' + MODEL_NAME + '_sup-bert-keras'

In [ ]:
OUTPUT_PATH = DATA_DIR + '/results/' + OUTPUT_NAME

In [ ]:
CACHE_DIR = DATA_DIR + 'cache'

In [ ]:
DF_SEP = "|"

In [ ]:
sent0_cname = "anchor"
sent1_cname = "target"
sent2_cname = "score"

# HyperParameters

In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2
labels = ["0.0", "0.25", "0.5", "0.75", "1.0"]
num_classes = len(labels)

# Dataset

In [ ]:
train_df = pd.read_csv(TRAIN_FILE, sep = DF_SEP)
valid_df = pd.read_csv(VALIDATION_FILE, sep = DF_SEP)
test_df = pd.read_csv(TEST_FILE, sep = DF_SEP)

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")


In [ ]:
print(f"Sentence1: {train_df.loc[1, sent0_cname]}")
print(f"Sentence2: {train_df.loc[1, sent1_cname]}")
print(f"Similarity: {train_df.loc[1, sent2_cname]}")

# Preprocessing

In [ ]:
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

In [ ]:
print("Train Target Distribution")
print(train_df[sent2_cname].value_counts())

In [ ]:
print("Validation Target Distribution")
print(valid_df[sent2_cname].value_counts())

In [ ]:
train_df["label"] = train_df[sent2_cname]
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=num_classes)

valid_df["label"] = valid_df[sent2_cname]
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=num_classes)


# Data Generator

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

# Model

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained(MODEL_NAME)
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(num_classes, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

# Batch

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[[sent0_cname, sent1_cname]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[[sent0_cname, sent1_cname]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

# Train

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

# Fine Tuning

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

# Train

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

# Save

In [ ]:
model.save(OUTPUT_PATH)

In [ ]:
%$%$^&

# Predict

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[[sent0_cname, sent1_cname]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

# Test

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[[sent0_cname, sent1_cname]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

# Inference

In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)